# Authors note

This file writes data_x (data) and data_y (labels) numpy files to be used to train machine learning algorithms. 

The data being written is generated from the 9 most chaotic food web models described as in "Food web complexity and chaotic population dynamics", by Fussmann and Heber (2002). The paper can be found here:
http://biology.mcgill.ca/faculty/fussmann/articles/FussmannHeber_02ELE.pdf.
The free parameters are the mortality rates of species z, y, x, c1, and c2, as  described in the aformentioned paper. 
Again, as described by Fussman and Heber combination of mortality rates which led to one or more species falling below the threshold, e, and therefor did not lead to coexistence of species were disregarded. 
Each time series was carried out 200,000 time steps into the future and the first 30,000 time steps were omitted from the recorded data. 


@author parkerkingfournier 


## Edited March 2021
I changed a few things and just made the data stored smaller, and also compatible with google drive etc.. Thank you to Parker Fournier for making his code legible and organized.

@editor Marie Cornellier

# Setup

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import numpy 	as np
from scipy import integrate
import pandas 	as pd
import math
import csv
from tqdm.notebook import tqdm
import random

In [4]:
def reduce_data(row, reduct):
  return row[::reduct]

# Math

## Global Variables

In [5]:
# Variables relating to functional response
a_1 = 0.125 	#Z -> Y
b_1 = 0.25
a_2 = 0.25		#Y -> X
b_2 = 0.5
a_3 = 1.0		#X -> Ci 	(i = 1,2)
b_3 = 2.0
a_4 = 7.5		#C1 -> P1 	&	C2 -> P3
b_4 = 5.0
a_5 = 0.25		#X -> Pi 	(i = 1,2,3)
b_5 = 0.5
a_6 = 2.5		#C1 -> P3	&	C2 -> P1
b_6 = 5.0
a_7 = 5.0		#Ci -> P2 	(i = 1,2)
b_7 = 5.0

# Intrinsic Growth Rate of Primary Producers
r = 2.5

# minimum value at which a population goes extinct
extinction = 0.000000001

## Ecologic Functions

In [6]:
# A type II saturating functional response
class ef:
  def f(u, v, a, b):
    # f_u = ((a*u)/(1+b*v))
    # if math.isnan(f_u):
      # f_u = np.nextafter(0,1)
    return ((a*u)/(1+b*v))

# Food Web Models

### three species chain


In [35]:
class threeSpeciesChain:
  def gen(file_x, file_y, file_p, length):
    # Three species food chain
    #		
    #		X
    #		|
    #		C
    #		|
    #		P
    #
    def dX_dt(t, X, d_x, d_c):
      x = X[0]
      c = X[1]
      p = X[2]
      """ Return the growth rate of all populations. """
      return np.array([ x*(ef.f(c,c,a_3,b_3) - d_x), 
                       c*(ef.f(p,p,a_4,b_4) - ef.f(x,X[1],a_3,b_3) - d_c), 
                       p*(r*(1 - p) - ef.f(c,p,a_4,b_4))])
    
    min_x, max_x = (0.01, 0.29)
    min_c, max_c = (0.04, 1.2)

    grid_fineness = 27

    num_models = grid_fineness * grid_fineness

    M = np.empty((3 * num_models,length))
    P = np.empty((num_models,2))

    index = 0
    p_index = 0
    # Iterate through mortality rates
    for d_x in tqdm(np.linspace(min_x, max_x, grid_fineness), desc="first loop"):
      for d_c in tqdm(np.linspace(min_c, max_c, grid_fineness), desc="second loop"):
        t = np.linspace(0, length,  length)
        X0 = np.array([0.5, 0.5, 0.5]) # initial conditions

        X = integrate.solve_ivp(dX_dt, [t[0], t[-1]], X0,t_eval=t,dense_output=True, args=(d_x, d_c))
        P1, P2, P3 = X.y[0],X.y[1],X.y[2]
        if len(P1) < length:
          P1 = np.concatenate((P1, np.repeat([0], length - len(P1))))
        if len(P2) < length:
          P2 = np.concatenate((P2, np.repeat([0], length - len(P2))))
        if len(P3) < length:
          P3 = np.concatenate((P3, np.repeat([0], length - len(P3))))
        M[index],M[index+1],M[index+2] = P1, P2, P3

        P[p_index][0] = d_x
        P[p_index][1] = d_c

        index += 3
        p_index += 1
      #d_c
    #d_x

    # M = np.apply_along_axis(reduce_data, 1, M, reduce_by)
    Y = np.repeat([2], M.shape[0])
    np.save(file_x, M, allow_pickle=True)
    np.save(file_y, Y, allow_pickle=True)
    np.save(file_p, P, allow_pickle=True)
  # end gen()
# end g2_chain

### Tritrophic Omnivory A (triangle)

In [27]:
class tritrophicOmnivoryA:
  def gen(file_x, file_y, file_p, length):
    # Food web with three trophic levels and an omnivorous predator
    #
    #			  X
    #		   /|
    #		  / |
    #		 C	| 
    #		  \ |
    #	     \|
    #			  P
    #
    def dX_dt(t, X, d_x, d_c):
      """ Return the growth rate of all populations. """
      x = X[0]
      c = X[1]
      p = X[2]
      return np.array([ x*((a_3*c + a_5*p)/(1 + b_3*c + b_5*p) - d_x), 
                       c*(ef.f(p,p,a_4,b_4) - (a_3*x)/(1 + b_3*c + b_5*p) - d_c), 
                       p*(r*(1-p) - ef.f(c,p,a_4,b_4) - (a_5*x)/(1 + b_5*p + b_3*c))])
    
    min_x, max_x = (0.14, 0.29)
    min_c, max_c = (0.04, 1.2)

    grid_fineness = 27

    num_models = grid_fineness * grid_fineness

    M = np.empty((3 * num_models,length))
    P = np.empty((num_models,2))

    index = 0
    p_index = 0
    # Iterate through mortality rates
    for d_x in tqdm(np.linspace(min_x, max_x, grid_fineness), desc="first loop"):
      for d_c in tqdm(np.linspace(min_c, max_c, grid_fineness), desc="second loop"):
        t = np.linspace(0, length,  length)
        X0 = np.array([0.5, 0.5, 0.5]) # initial conditions

        X = integrate.solve_ivp(dX_dt, [t[0], t[-1]], X0,t_eval=t,dense_output=True, args=(d_x, d_c))
        P1, P2, P3 = X.y[0],X.y[1],X.y[2]
        if len(P1) < length:
          P1 = np.concatenate((P1, np.repeat([0], length - len(P1))))
        if len(P2) < length:
          P2 = np.concatenate((P2, np.repeat([0], length - len(P2))))
        if len(P3) < length:
          P3 = np.concatenate((P3, np.repeat([0], length - len(P3))))
        M[index],M[index+1],M[index+2] = P1, P2, P3

        P[p_index][0] = d_x
        P[p_index][1] = d_c

        index += 3
        p_index += 1
      #d_c
    #d_x

    # M = np.apply_along_axis(reduce_data, 1, M, reduce_by)
    Y = np.repeat([3], M.shape[0])
    np.save(file_x, M, allow_pickle=True)
    np.save(file_y, Y, allow_pickle=True)
    np.save(file_p, P, allow_pickle=True)
  # end gen()
# end g2_a

### Tritrophic Omnivory B 

In [28]:
class tritrophicOmnivoryB:
  def gen(file_x, file_y, file_p, length):
    # Food web with three trophic levels and an omnivorous predator
    #
    #			  X
    #		   /|\
    #		  / | \
    #		 C1	|  C2
    #		  \ | /
    #	     \|/
    #			  P
    #
    def dX_dt(t, X, d_x, d_c1, d_c2):
      x = X[0]
      c1 = X[1]
      c2 = X[2]
      p = X[3]
      """ Return the growth rate of all populations. """
      return np.array([ x*((a_3*c1 + a_3*c2 + a_5*p)/(1 + b_3*c1 + b_3*c2 + b_5*p) - d_x),
                       c1*(ef.f(p,p,a_4,b_4) - ef.f(x,c1,a_3,b_3) - d_c1),
                       c2*(ef.f(p,p,a_4,b_4) - ef.f(x,c2,a_3,b_3) - d_c2),
                       p*(r*(1-p) - ef.f(c1,p,a_4,b_4) - ef.f(c2,p,a_4,b_4) - (a_5*x)/(1 + b_5*p + b_3*c2))])
    
    min_x, max_x = (0.21, 0.25)
    min_c, max_c = (0.54, 1.12)

    grid_fineness = 27

    num_models = grid_fineness * grid_fineness

    M = np.empty((4 * num_models,length))
    P = np.empty((num_models,2))

    index = 0
    p_index = 0
    # Iterate through mortality rates
    for d_x in tqdm(np.linspace(min_x, max_x, grid_fineness), desc="first loop"):
      for d_c in np.linspace(min_c, max_c, grid_fineness):
        t = np.linspace(0, length,  length)
        X0 = np.array([0.5, 0.5, 0.5, 0.5]) # initial conditions

        X = integrate.solve_ivp(dX_dt, [t[0], t[-1]], X0,t_eval=t,dense_output=True, args=(d_x, d_c, d_c))
        P1, P2, P3, P4 = X.y[0],X.y[1],X.y[2],X.y[3]
        if len(P1) < length:
          P1 = np.concatenate((P1, np.repeat([0], length - len(P1))))
        if len(P2) < length:
          P2 = np.concatenate((P2, np.repeat([0], length - len(P2))))
        if len(P3) < length:
          P3 = np.concatenate((P3, np.repeat([0], length - len(P3))))
        if len(P4) < length:
          P4 = np.concatenate((P4, np.repeat([0], length - len(P4))))
        
        M[index],M[index+1],M[index+2],M[index+3] = P1, P2, P3, P4

        P[p_index][0] = d_x
        P[p_index][1] = d_c

        index += 4
        p_index += 1
      #d_c
    #d_x

    # M = np.apply_along_axis(reduce_data, 1, M, reduce_by)
    Y = np.repeat([4], M.shape[0])
    np.save(file_x, M, allow_pickle=True)
    np.save(file_y, Y, allow_pickle=True)
    np.save(file_p, P, allow_pickle=True)

  # end gen()
# end g3_b

### Tritrophic Omnivory C

In [29]:
class tritrophicOmnivoryC:
  def gen(file_x, file_y, file_p, length):
    # Food web with three trophic levels and an omnivorous predator
    #
    #	--------X--------
    #	|	     / \      |
    #	|	    /   \     |
    #	|	   /     \    |
    #	|   C1      C2  |
    #	|   |\	    /|  |
    #	|   | \    / |  |
    # |   |  \  /  |  |  
    #	|   |   \/	 |  |
    # |   |   /\   |  |
    # |   |  /  \  |  |
    # |   | /    \ |  |
    #	|--P1       P2--|
    #
    def dX_dt(t, X, d_x, d_c1, d_c2):
      x = X[0]
      c1 = X[1]
      c2 = X[2]
      p1 = X[3]
      p2 = X[4]
      """ Return the growth rate of all populations. """
      return np.array([ x*((a_3*c1 + a_3*c2 + a_5*p1 + a_5*p2)/(1 + b_3*c1 + b_3*c2 + b_5*p1 + b_5*p1) - d_x), 
                       c1*((a_4*p1 + a_6*p1)/(1+b_4*p1 + b_6*p2) - (a_3*x)/(1 + b_3*c1 + b_3*c2 + b_5*p1 + b_5*p2) - d_c1), 
                       c2*((a_6*p1 + a_2*p2)/(1+b_6*p1 + b_4*p2) - (a_3*x)/(1 + b_3*c1 + b_3*c2 + b_5*p1 + b_5*p2) - d_c2),
                       p1*(r*(1-p1) - (a_4*c1)/(1 + b_4*p1 + b_6*p2) - (a_6*c2)/(1 + b_6*p1 + b_4*p2) - (a_5*x)/(1 + b_5*p1 + b_5*p2 + b_3*c1 + b_3*c2)),
                       p2*(r*(1-p2) - (a_6*c1)/(1 + b_6*p1 + b_4*p2) - (a_4*c2)/(1 + b_4*p1 + b_6*p2) - (a_5*x)/(1 + b_5*p1 + b_5*p2 + b_3*c1 + b_3*c2))])

    min_x, max_x = (0.26, 0.42)
    min_c1, max_c1 = (0.51, 0.87)
    min_c2, max_c2 = (0.01, 0.27)

    grid_fineness = 7

    num_models = grid_fineness * grid_fineness * grid_fineness

    M = np.empty((5 * num_models,length))
    P = np.empty((num_models,3))

    index = 0
    p_index = 0
    # Iterate through mortality rates
    for d_x in tqdm(np.linspace(min_x, max_x, grid_fineness)):
      for d_c1 in np.linspace(min_c1, max_c1, grid_fineness):
        for d_c2 in np.linspace(min_c2, max_c2, grid_fineness):
          t = np.linspace(0, length,  length)
          X0 = np.array([0.5, 0.5, 0.5, 0.5, 0.5]) # initial conditions

          X = integrate.solve_ivp(dX_dt, [t[0], t[-1]], X0,t_eval=t,dense_output=True, args=(d_x, d_c1, d_c2))
          P1, P2, P3, P4, P5 = X.y[0],X.y[1],X.y[2],X.y[3],X.y[4]
          if len(P1) < length:
            P1 = np.concatenate((P1, np.repeat([0], length - len(P1))))
          if len(P2) < length:
            P2 = np.concatenate((P2, np.repeat([0], length - len(P2))))
          if len(P3) < length:
            P3 = np.concatenate((P3, np.repeat([0], length - len(P3))))
          if len(P4) < length:
            P4 = np.concatenate((P4, np.repeat([0], length - len(P4))))
          if len(P5) < length:
            P5 = np.concatenate((P5, np.repeat([0], length - len(P5))))
          
          M[index],M[index+1],M[index+2],M[index+3],M[index+4] = P1, P2, P3, P4, P5

          P[p_index][0] = d_x
          P[p_index][1] = d_c1
          P[p_index][2] = d_c2

          index += 5
          p_index += 1
        #d_c2
      #d_c1
    #d_x

    # M = np.apply_along_axis(reduce_data, 1, M, reduce_by)
    Y = np.repeat([5], M.shape[0])
    np.save(file_x, M, allow_pickle=True)
    np.save(file_y, Y, allow_pickle=True)
    np.save(file_p, P, allow_pickle=True)
  # end gen()
# end g3_c

### Four species chain

In [30]:
class fourSpeciesChain:
  def gen(file_x, file_y, file_p, length):
    # Four spieces food chain
    # 		
    #		Y
    #		|
    #		X
    #		|
    #		C
    #		|
    #		P
    #
    def dX_dt(t, X, d_x, d_c):
      y = X[0]
      x = X[1]
      c = X[2]
      p = X[3]
      """ Return the growth rate of all populations. """
      return np.array([ y*(ef.f(x,x,a_2,b_2) - d_y), 
                       x*(ef.f(c,c,a_3,b_3) - ef.f(X[0],x,a_2,b_2) - d_x), 
                       c*(ef.f(p,p,a_4,b_4) - ef.f(x,c,a_3,b_3) - d_c),
                       p*(r*(1 - p) 	  - ef.f(c,p,a_4,b_4))])

    min_y, max_y = (0.01, 0.14)
    min_x, max_x = (0.01, 0.09)
    min_c, max_c = (0.01, 0.19)

    grid_fineness = 7

    num_models = grid_fineness * grid_fineness * grid_fineness

    M = np.empty((4 * num_models,length))
    P = np.empty((num_models,3))

    index = 0
    p_index = 0

    # Iterate through mortality rates
    for d_y in np.linspace(min_y, max_y, grid_fineness):
      for d_x in np.linspace(min_x, max_x, grid_fineness):
        for d_c in np.linspace(min_c, max_c, grid_fineness):
          t = np.linspace(0, length,  length)
          X0 = np.array([0.5, 0.5, 0.5, 0.5]) # initial conditions

          X = integrate.solve_ivp(dX_dt, [t[0], t[-1]], X0,t_eval=t,dense_output=True, args=(d_x, d_c))
          P1, P2, P3, P4 = X.y[0],X.y[1],X.y[2],X.y[3]
          if len(P1) < length:
            P1 = np.concatenate((P1, np.repeat([0], length - len(P1))))
          if len(P2) < length:
            P2 = np.concatenate((P2, np.repeat([0], length - len(P2))))
          if len(P3) < length:
            P3 = np.concatenate((P3, np.repeat([0], length - len(P3))))
          if len(P4) < length:
            P4 = np.concatenate((P4, np.repeat([0], length - len(P4))))
          
          M[index],M[index+1],M[index+2],M[index+3] = P1, P2, P3, P4

          P[p_index][0] = d_x
          P[p_index][1] = d_c

          index += 4
          p_index += 1
        #d_c
      #d_x
    #d_y
    # M = np.apply_along_axis(reduce_data, 1, M, reduce_by)
    Y = np.repeat([1], M.shape[0])
    np.save(file_x, M, allow_pickle=True)
    np.save(file_y, Y, allow_pickle=True)
    np.save(file_p, P, allow_pickle=True)
  # end gen()
# end g3_chain

### Tritrophic No Omnivory

In [31]:
class tritrophicNoOmnivory:
  def gen(file_x, file_y, file_p, length):
    # Food web with three trophic levels and an omnivorous predator
    #
    #	        X       
    #	 	     / \      
    #	 	    /   \     
    #		   /     \    
    #	    C1      C2  
    #	    |\	    /|  
    #	    | \    / |  
    #     |  \  /  |  
    #	    |   \/	 |  
    #     |   /\   |  
    #     |  /  \  |  
    #     | /    \ |  
    #	    P1      P2
    #
    def dX_dt(t, X, d_x, d_c1, d_c2):
      x = X[0]
      c1 = X[1]
      c2 = X[2]
      p1 = X[3]
      p2 = X[4]
      """ Return the growth rate of all populations. """
      return np.array([ x*((a_3*c1 + a_3*c2 + a_5*p1)/(1 + b_3*c1 + b_3*c2 + b_5*p1) - d_x), 
                       c1*((a_4*p1 + a_6*p2)/(1+b_4*p1 + b_6*p2) - (a_3*x)/(1 + b_3*c1 + b_3*c2) - d_c1),
                       c2*((a_6*p1 + a_2*p2)/(1+b_6*p1 + b_4*p2) - (a_3*x)/(1 + b_3*c1 + b_3*c2) - d_c2),
                       p1*(r*(1-p1) - ef.f(c1,p1,a_4,b_4) - ef.f(c2,p1,a_6,b_6)),
                       p2*(r*(1-p2) - ef.f(c1,p2,a_6,b_6) - ef.f(c2,p2,a_4,b_4))])

    min_x, max_x = (0.22, 0.30)
    min_c1, max_c1 = (0.68, 0.78)
    min_c2, max_c2 = (0.09, 0.19)

    grid_fineness = 7

    num_models = grid_fineness * grid_fineness * grid_fineness

    M = np.empty((5 * num_models,length))
    P = np.empty((num_models,3))

    index = 0
    p_index = 0
    # Iterate through mortality rates
    for d_x in tqdm(np.linspace(min_x, max_x, grid_fineness)):
      for d_c1 in np.linspace(min_c1, max_c1, grid_fineness):
        for d_c2 in np.linspace(min_c2, max_c2, grid_fineness):
          t = np.linspace(0, length,  length)
          X0 = np.array([0.5, 0.5, 0.5, 0.5, 0.5]) # initial conditions

          X = integrate.solve_ivp(dX_dt, [t[0], t[-1]], X0,t_eval=t,dense_output=True, args=(d_x, d_c1, d_c2))
          P1, P2, P3, P4, P5 = X.y[0],X.y[1],X.y[2],X.y[3],X.y[4]
          if len(P1) < length:
            P1 = np.concatenate((P1, np.repeat([0], length - len(P1))))
          if len(P2) < length:
            P2 = np.concatenate((P2, np.repeat([0], length - len(P2))))
          if len(P3) < length:
            P3 = np.concatenate((P3, np.repeat([0], length - len(P3))))
          if len(P4) < length:
            P4 = np.concatenate((P4, np.repeat([0], length - len(P4))))
          if len(P5) < length:
            P5 = np.concatenate((P5, np.repeat([0], length - len(P5))))
          
          M[index],M[index+1],M[index+2],M[index+3],M[index+4] = P1, P2, P3, P4, P5

          P[p_index][0] = d_x
          P[p_index][1] = d_c1
          P[p_index][2] = d_c2

          index += 5
          p_index += 1
        #d_c2
      #d_c1
    #d_x

    # M = np.apply_along_axis(reduce_data, 1, M, reduce_by)
    Y = np.repeat([7], M.shape[0])
    np.save(file_x, M, allow_pickle=True)
    np.save(file_y, Y, allow_pickle=True)
    np.save(file_p, P, allow_pickle=True)
  # end gen()
# end g3_c

### Tritrophic Partial Omnivory

In [32]:
class tritrophicPartialOmnivory:
  def gen(file_x, file_y, file_p, length):
    # Food web with three trophic levels and a semi-omnivorous predator
    #
    #	--------X
    #	|	     / \
    #	|	    /   \
    #	|	   /     \	
    #	|   C1      C2
    #	|   |\	    /|
    #	|   | \    / |
    # |   |  \  /  |
    #	|   |   \/	 |
    # |   |   /\   |
    # |   |  /  \  |
    # |   | /    \ |
    #	|--P1       P2
    #
    def dX_dt(t, X, d_x, d_c1, d_c2):
      x = X[0]
      c1 = X[1]
      c2 = X[2]
      p1 = X[3]
      p2 = X[4]
      """ Return the growth rate of all populations. """
      return np.array([ x*((a_3*c1 + a_3*c2 + a_5*p1)/(1 + b_3*c1 + b_3*c2 + b_5*p1) - d_x), 
                       c1*((a_4*p1 + a_6*p2)/(1+b_4*p1 + b_6*p2) - (a_3*x)/(1 + b_3*c1 + b_3*c2 + b_5*p1) - d_c1),
                       c2*((a_6*p1 + a_2*p2)/(1+b_6*p1 + b_4*p2) - (a_3*x)/(1 + b_3*c1 + b_3*c2 + b_5*p1) - d_c2),
                       p1*(r*(1-p1) - ef.f(c1,p1,a_6,b_6) - ef.f(c2,p1,a_6,b_6) - (a_5*x)/(1 + b_5*p1 + b_3*p1)),
                       p2*(r*(1-p2) - ef.f(c1,p2,a_6,b_6) - ef.f(c2,p2,a_6,b_6) - (a_5*x)/(1 + b_5*p1 + b_3*p1))])

    min_x, max_x = (0.17, 0.39)
    min_c1, max_c1 = (0.3, 0.87)
    min_c2, max_c2 = (0.01, 0.27)

    grid_fineness = 5

    num_models = grid_fineness * grid_fineness * grid_fineness

    M = np.empty((5 * num_models,length))
    P = np.empty((num_models,3))

    index = 0
    p_index = 0
    # Iterate through mortality rates
    for d_x in tqdm(np.linspace(min_x, max_x, grid_fineness)):
      for d_c1 in tqdm(np.linspace(min_c1, max_c1, grid_fineness)):
        for d_c2 in tqdm(np.linspace(min_c2, max_c2, grid_fineness)):
          t = np.linspace(0, length,  length)
          X0 = np.array([0.5, 0.5, 0.5, 0.5, 0.5]) # initial conditions

          X = integrate.solve_ivp(dX_dt, [t[0], t[-1]], X0,t_eval=t,dense_output=True, args=(d_x, d_c1, d_c2))
          P1, P2, P3, P4, P5 = X.y[0],X.y[1],X.y[2],X.y[3],X.y[4]
          if len(P1) < length:
            P1 = np.concatenate((P1, np.repeat([0], length - len(P1))))
          if len(P2) < length:
            P2 = np.concatenate((P2, np.repeat([0], length - len(P2))))
          if len(P3) < length:
            P3 = np.concatenate((P3, np.repeat([0], length - len(P3))))
          if len(P4) < length:
            P4 = np.concatenate((P4, np.repeat([0], length - len(P4))))
          if len(P5) < length:
            P5 = np.concatenate((P5, np.repeat([0], length - len(P5))))
          
          M[index],M[index+1],M[index+2],M[index+3],M[index+4] = P1, P2, P3, P4, P5

          P[p_index][0] = d_x
          P[p_index][1] = d_c1
          P[p_index][2] = d_c2

          index += 5
          p_index += 1
        #d_c2
      #d_c1
    #d_x

    # M = np.apply_along_axis(reduce_data, 1, M, reduce_by)
    Y = np.repeat([6], M.shape[0])
    np.save(file_x, M, allow_pickle=True)
    np.save(file_y, Y, allow_pickle=True)
    np.save(file_p, P, allow_pickle=True)
  # end gen()
# end g3_c

### Five Species Chain

In [33]:
class fiveSpeciesChain:
  def gen(file_x, file_y, file_p, length):
    # Five spieces food chain
    #		
    #		Z
    #		|
    #		Y
    #		|
    #		X
    #		|
    #		C
    #		|
    #		P
    #
    def dX_dt(t, X, d_z, d_y, d_x, d_c):
      z = X[0]
      y = X[1]
      x = X[2]
      c = X[3]
      p = X[4]
      """ Return the growth rate of all populations. """
      return np.array([ z*(ef.f(y,y,a_1,b_1) - d_z), 
                       y*(ef.f(x,x,a_2,b_2) - ef.f(X[0],y,a_1,b_1) - d_y),
                       x*(ef.f(c,c,a_3,b_3) - ef.f(y,x,a_2,b_2) - d_x),
                       c*(ef.f(p,p,a_4,b_4) - ef.f(x,c,a_3,b_3) - d_c),
                       p*(r*(1 - p) - ef.f(c,p,a_4,b_4))])

    d_z = 0.01
    min_y, max_y = (0.01, 0.09)
    min_x, max_x = (0.01, 0.11)
    min_c, max_c = (0.01, 0.11)

    grid_fineness = 7

    num_models = grid_fineness * grid_fineness * grid_fineness

    M = np.empty((5 * num_models,length))
    P = np.empty((num_models,4))

    index = 0
    p_index = 0
    # Iterate through mortality rates
    for d_y in tqdm(np.linspace(min_y, max_y, grid_fineness)):
      for d_x in tqdm(np.linspace(min_x, max_x, grid_fineness)):
        for d_c in tqdm(np.linspace(min_c, max_c, grid_fineness)):
          t = np.linspace(0, length,  length)
          X0 = np.array([0.5, 0.5, 0.5, 0.5, 0.5]) # initial conditions

          X = integrate.solve_ivp(dX_dt, [t[0], t[-1]], X0,t_eval=t,dense_output=True, args=(d_z, d_y, d_x, d_c))
          P1, P2, P3, P4, P5 = X.y[0],X.y[1],X.y[2],X.y[3],X.y[4]
          if len(P1) < length:
            P1 = np.concatenate((P1, np.repeat([0], length - len(P1))))
          if len(P2) < length:
            P2 = np.concatenate((P2, np.repeat([0], length - len(P2))))
          if len(P3) < length:
            P3 = np.concatenate((P3, np.repeat([0], length - len(P3))))
          if len(P4) < length:
            P4 = np.concatenate((P4, np.repeat([0], length - len(P4))))
          if len(P5) < length:
            P5 = np.concatenate((P5, np.repeat([0], length - len(P5))))
          
          M[index],M[index+1],M[index+2],M[index+3],M[index+4] = P1, P2, P3, P4, P5

          P[p_index][0] = d_z
          P[p_index][1] = d_y
          P[p_index][2] = d_x
          P[p_index][3] = d_c

          index += 5
          p_index += 1
        #d_c2
      #d_c1
    #d_x

    # M = np.apply_along_axis(reduce_data, 1, M, reduce_by)
    Y = np.repeat([0], M.shape[0])
    np.save(file_x, M, allow_pickle=True)
    np.save(file_y, Y, allow_pickle=True)
    np.save(file_p, P, allow_pickle=True)
  # end gen()
# end g3_c

# Generate and Write Data

In [36]:
# Length describes how far out to run each model in the future 
# if you change dt and the length accordingly you can adjust the overall resolution of the data
# it is helpful to keep the resolution low while initially playing with the data because it saves a lot of time and memory
length = 4000

# reduce data factor
# reduce_by = 1

# time step
dt = 0.01
# see if non-extinction is extended with smaller time step

for i in range(0, 1):
  file_x = '/content/drive/MyDrive/Food-Web-Project/Data/data_x_' + str(i) + '.npy'
  file_y = '/content/drive/MyDrive/Food-Web-Project/Data/data_y_' + str(i) + '.npy'
  file_p = '/content/drive/MyDrive/Food-Web-Project/Params/' + str(i) + '.npy'
  print("\nGenerating data for model number:", i)
  if i == 0:
    threeSpeciesChain.gen(	  file_x, file_y, file_p, length)
  elif i == 1:
    tritrophicOmnivoryA.gen(	  file_x, file_y, file_p, length)
  elif i == 2:
    tritrophicOmnivoryB.gen(	  file_x, file_y, file_p, length)
  elif i == 3:
    tritrophicOmnivoryC.gen(	  file_x, file_y, file_p, length)
  elif i == 4:
    fourSpeciesChain.gen(	  file_x, file_y, file_p, length)
  elif i == 5:
    tritrophicNoOmnivory.gen(		file_x, file_y, file_p, length)
  elif i == 6:
    tritrophicPartialOmnivory.gen(		file_x, file_y, file_p, length)
  elif i == 7:
    fiveSpeciesChain.gen(			file_x, file_y, file_p, length)
  print("\n	Finished!")
  print("	Training data written to ", file_x, " and labels written to ", file_y)
  print("	\nParams written to ", file_p)


Generating data for model number: 0


KeyboardInterrupt: ignored